In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Four in a row Agent and Environment

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis vestibulum, nibh sit amet convallis varius, neque arcu luctus nunc, gravida sollicitudin quam nisi mattis ante. Duis imperdiet lorem a pharetra iaculis. In hac habitasse platea dictumst. Vivamus nisl ex, semper eget facilisis vel, varius ac est. Curabitur eget nisl erat. In egestas molestie mi, a pulvinar ante sodales sit amet. Vivamus interdum sem eu interdum placerat. Duis tempus purus non lectus pharetra, ac gravida mauris ultricies. Vestibulum condimentum ut dolor nec interdum. Curabitur et ex magna. In erat purus, laoreet vitae elementum sed, vehicula eu lorem. Nulla maximus eu est vel tempor. Ut et aliquam quam, eu pellentesque leo. Ut at suscipit eros. Phasellus luctus, justo nec molestie pharetra, massa sem hendrerit arcu, ut ullamcorper justo augue et est. Donec dictum sem id tristique pharetra.

#TODO state which libaries are used 

## 1. Definition of the Environment

- States: The environment is in one of the following states $S = (S_0, S_1, ..., S_w)$ with $S_x = (S_{x0}, S_{x1}, ..., S_{xh})$ and $S_{xy} \in \{E, R, Y\}$.
- Actions: The set of  available actions: $a \in \{n \in \mathbb{N}: (|E \in S_n| > 0)\}$ (the agent can place their chip in any non-full column).
- Transitions: The transition depends on the opponent's strategy. If we assume the opponent plays randomly then the probability of them picking a viable column is $\dfrac{1}{nr\;of\;valid\;columns}$

The `FourInARowEnv` class has the following methods:
- `reset()` resets the environment's state to it's initial state.
- `step(action)` processes the action of the agent.
- `render()` displays the state using box characters.

To allow an agent to calculate optimal decisions using model information, these methods are also available:
- `get_possible_states()` calculates all possible future states.
- `is_done()` checks if there is a winner of if the board is full.
- `get_reward_for_new(state)` simplified version $R(s)$ of the general reward function: $R(s, a, s')$.
- `get_transition_prob(action, new_state, old_state)` $P(s' \mid s, a)$.


## 2. Environment Creation

We are able to create an environment with several parameters:

- `yellow_agent` is used to define an agent that will act as the opponent.
- `width` the width of the playing field.
- `height` the height of the playing field.
- `win_conditions` the number of chips in a row required to win.
- `first_turn` the player that will play first.


In [64]:
from src import FourInARowEnv, FourInARowRandomAgent, Players, FourInARowRenderer

In [65]:
environment = FourInARowEnv(
  yellow_agent  = FourInARowRandomAgent,
  width         = 3,
  height        = 3,
  win_condition = 4,
  first_turn    = Players.RED
)

### 2.1 Displaying the board

The board is displayed using box characters. The letter `Y` represents a yellow chip, and the letter `R` represents a red one.

In [66]:
red_agent = FourInARowRandomAgent(environment)

environment.step(red_agent.get_move())

print(environment.render())

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│   │   │ Y │
├───┼───┼───┤
│   │   │ R │
└───┴───┴───┘


### 2.2 Possible states and actions



In [67]:
environment.reset()

print(environment.get_possible_actions())
for state in environment.get_possible_states()[0:3]:
  print(FourInARowRenderer(state).render())

[0, 1, 2]
┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│   │   │   │
├───┼───┼───┤
│ R │   │   │
└───┴───┴───┘
┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ Y │   │   │
├───┼───┼───┤
│ R │   │   │
└───┴───┴───┘
┌───┬───┬───┐
│ R │   │   │
├───┼───┼───┤
│ Y │   │   │
├───┼───┼───┤
│ R │   │   │
└───┴───┴───┘


# 4. Value Iteration

Value Iteration is based on the Bellman update:

(6) $U_{i+1}(s) = \underset{a}{max} \sum_{s'} P(s' \mid s, a) \space [ R(s, a, s') + \gamma \space U_i(s') ]$

Using equation (3) this simplifies to:

(7) $U_{i+1}(s) = \underset{a}{max} \space Q_i(s, a)$

One can prove that after enough iterations $U_{i+1}(s) \approx U(s)$, after which Bellman's equation is satisfied.  
Since there is only one solution to Bellman's equation, it does not matter with which $U_0(s)$ you start!

The algorithm below is Value Iteration with one simplification: $\gamma$ the so-called discount factor, is set to 1.


help....

In [68]:
def get_initial_U(mdp):
    U = {}
    for s in mdp.get_possible_states():
        U[s] = mdp.get_reward_for_new(s)
    return U
    
def Q_Value(mdp, s, a, U):
    Q = 0.0
    for s_p in mdp.get_possible_states():
        P = mdp.get_transition_prob(a, s_p, s)
        R = mdp.get_reward_for_new(s_p)
        Q += P * (R + U[tuple(s_p)])
    return Q

def ValueIteration(mdp, error=0.00001):
    # from AIMA 4th edition without discount gamma 
    U_p = get_initial_U(mdp) # U_p = U'
    delta = float('inf')
    while delta > error:
        U = {}
        for s in mdp.get_possible_states():
            U[s] = U_p[s]
        #print_U(U)  # to illustrate the iteration process
        delta = 0
        for s in mdp.get_possible_states():
            max_a = float('-inf')
            for a in mdp.get_possible_actions(s):
                q = Q_Value(mdp, s, a, U) 
                if q > max_a:
                    max_a = q
            U_p[s] = max_a
            if abs(U_p[s] - U[s]) > delta:
                delta = abs(U_p[s] - U[s])
    return U

ValueIteration(environment)

KeyError: <src.FourInARowState.FourInARowState object at 0x000002E21D5F0880>

In [ ]:
print(environment.is_done())

True
